# London Crime Prediction - location and crime type

This is my research on **London Metropolitian** crimes in the UK based on a 1 year period from 2019-2020 using data supplied by [Police data UK](https://data.police.uk/).  It is for interest purposes only and also my fascination with being able to use a machine to try and predict the location of a next event.  It includes some very basic analysis and maping.  The main importance for me is to train very accurate models on next type of crime and location prediction.  I welcome any feedback and advise as this is very much an ongoing project.

First we load dependancies and make the input data files accessible. 

In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from glob import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_absolute_error
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium.plugins import FastMarkerCluster


import math


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
files = []
import os
for dirname, _, filenames in os.walk('/home/dean/Downloads/crime_data'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
print(files)        
# Any results you write to the current directory are saved as output.

[]


Inspecting the first 2 input files we can see the data columns structures are the same so we can concantenate all the files to create one dataset.

In [61]:
df1 = pd.read_csv(files[0])
df2 = pd.read_csv(files[1])


print(df1.head)
print(df2.head)

IndexError: list index out of range

Cleaning up the filenames and adding to an iterable list using the following code.

In [33]:
dataframes = [pd.read_csv(file).assign(crime_file=os.path.basename(file).strip(".csv")) for file in files]

Now to concatenate all the files to produce 1 dataset.

In [34]:
data_full = pd.concat(dataframes, ignore_index=True)


ValueError: No objects to concatenate

Now to inspect the concatenation.

In [35]:
data_full.shape[0]

NameError: name 'data_full' is not defined

In [36]:
data_full.head()

NameError: name 'data_full' is not defined

In [37]:
data_full.tail(13000)

NameError: name 'data_full' is not defined

In [38]:
data_full.sample(50)

NameError: name 'data_full' is not defined

Checking for missing data and also printing the percentage of missing latitude and longitude positions.

In [39]:
missing_values_count = data_full.isnull().sum()
print(missing_values_count)

NameError: name 'data_full' is not defined

In [40]:
print("Percentage of missing latitude and longitude values:(%.1f%%)" %(missing_values_count['Latitude']/data_full.shape[0]*100))

NameError: name 'missing_values_count' is not defined

Next to clean up the `missing_values_count`. With the missing lattitude and longitude values at only 1.14% I have elected to input the latitude and longitude values as central London so as to keep the type of crime figures with only marginal error in location.

In [41]:
data_full[pd.isnull(data_full['LSOA name'])].shape


NameError: name 'data_full' is not defined

On further analysis the columns with `NaN` on `'Longitude'` and `'Latitude'` are also without `'LSOA name'` or the crime was resolved at scene or no other information.  Droping rows with NaN for `'LOSA name'`

In [42]:
location_nan_clean = data_full.dropna(axis=0, subset=['LSOA name'])

NameError: name 'data_full' is not defined

In [43]:
location_nan_clean.sample(70)
data = location_nan_clean.reset_index()

NameError: name 'location_nan_clean' is not defined

Checking the `tail` of the dataset to see if the `NaN` locations have been removed.

In [44]:
data.tail()

NameError: name 'data' is not defined

Plotting data so far....

In [45]:
# Set the width and height of the figure
plt.figure(figsize=(8,10))

# Add title
plt.title("Number of metropolitain crimes by type")

# Y-axis crime types
sns.barplot(x=data.index,y=data['Crime type'])

# Add label for vertical axis
plt.ylabel("Type of crime")




NameError: name 'data' is not defined

Plotting line graph for type of crime over date committed.

In [46]:
data.dtypes


NameError: name 'data' is not defined

Create month column for month number to create line graph.

In [47]:
data['month'] = pd.DatetimeIndex(data['Month']).month



NameError: name 'data' is not defined

In [48]:
data.head()

NameError: name 'data' is not defined

In [49]:
data.tail()

NameError: name 'data' is not defined

Task now to hot encode crime types to numerical values.

In [50]:
# Get list of categorical variables
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

NameError: name 'data' is not defined

In [51]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if data[col].nunique() < 15]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

NameError: name 'object_cols' is not defined

In [52]:
data.head()

NameError: name 'data' is not defined

In [53]:
cat_crime = pd.get_dummies(data['Crime type'])



cat_crime.Drugs.value_counts()

NameError: name 'data' is not defined

In [54]:
cat_crime.head()

NameError: name 'cat_crime' is not defined

In [55]:
merged = pd.concat([data, cat_crime], axis='columns')

merged.head()

NameError: name 'data' is not defined

In [56]:
for col in merged.columns: 
    print(col)

NameError: name 'merged' is not defined

In [57]:
final = merged.drop(['Crime type', 'month', 'LSOA code','LSOA name', 'crime_file', 'Crime ID', 'Reported by', 'Falls within', 'Last outcome category', 'Context','Location'], axis='columns')

final.reset_index

final.head(50)

NameError: name 'merged' is not defined

In [58]:
# Set the width and height of the figure
plt.figure(figsize=(14,6))

# Add title
plt.title("Burglary and drugs 2019-2020")



sns.lineplot(x=final['Month'],y=final['Burglary'], label="Burglary")

sns.lineplot(x=final['Month'],y=final['Drugs'], label="Drugs")


# Add label for horizontal axis
plt.xlabel("Month")

NameError: name 'final' is not defined

In [59]:
m_1 = folium.Map(location=[51.4203,0.0705], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=final[['Latitude', 'Longitude','Violence and sexual offences']], radius=10).add_to(m_1)

# Display the map
m_1

NameError: name 'final' is not defined

In [60]:
# Create the map
m_3 = folium.Map(location=[51.4203,0.0705], tiles='cartodbpositron', zoom_start=11)

final_sample = final.sample(n=50000)
# Add points to the map
mc = MarkerCluster()



for idx, row in final_sample.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker([row['Latitude'], row['Longitude']],popup=(row == 1).idxmax(axis=1)))

m_3.add_child(mc)




# Display the map
m_3

NameError: name 'final' is not defined